# Backtesting ML Classification-Based

In [ ]:
???

![](src/res_classification.png)

## Load the model

In [1]:
import pickle

In [2]:
with open('models/model_dt_classification.pkl', 'rb') as f:
    model_dt = pickle.load(f)

In [3]:
model_dt

DecisionTreeClassifier(max_depth=20)

## Load the data

In [4]:
import pandas as pd

df = pd.read_excel('data/Microsoft_LinkedIn_Processed.xlsx', index_col=0, parse_dates=['Date'])
df

,Open,High,Low,Close,Volume,change_tommorow,change_tommorow_direction
Date,,,,,,,
2016-12-08,61.299999,61.580002,60.840000,61.009998,21220800,1.549141,UP
2016-12-09,61.180000,61.990002,61.130001,61.970001,27349400,0.321694,UP
2016-12-12,61.820000,62.299999,61.720001,62.169998,20198100,1.286125,UP
2016-12-13,62.500000,63.419998,62.240002,62.980000,35718900,-0.478620,DOWN
2016-12-14,63.000000,63.450001,62.529999,62.680000,30352700,-0.159793,DOWN
...,...,...,...,...,...,...,...
2024-07-23,443.899994,448.390015,443.100006,444.850006,13107100,-3.718818,DOWN
2024-07-24,440.450012,441.480011,427.589996,428.899994,26805800,-2.509560,DOWN
2024-07-25,428.799988,429.799988,417.510010,418.399994,29943800,1.615443,UP


## Backtesting.py Library

### Create your Strategy Class

In [5]:
from backtesting import Backtest, Strategy

In [6]:
df_explanatory = df.drop(columns=['change_tommorow', 'change_tommorow_direction'])

In [7]:
model_dt.predict(X=df_explanatory)

array(['UP', 'UP', 'UP', ..., 'UP', 'UP', 'DOWN'], dtype=object)

In [9]:
explanatory_today = df_explanatory.iloc[[-1],:]
explanatory_today

,Open,High,Low,Close,Volume
Date,,,,,
2024-07-29,431.579987,432.149994,424.700012,426.730011,15125800


In [10]:
forecast_tommorow = model_dt.predict(explanatory_today)[0]

#### Calculate Purchase Recommendation

##### Buy if it goes up

In [13]:
list_buy_sell = []

In [14]:
for tommorow_direction in df.change_tommorow_direction:
  if tommorow_direction == 'UP':
    list_buy_sell.append(1)
  elif tommorow_direction == 'DOWN':
    list_buy_sell.append(-1)

In [17]:
df['buy_sell'] = list_buy_sell
df

,Open,High,Low,Close,Volume,change_tommorow,change_tommorow_direction,buy_sell
Date,,,,,,,,
2016-12-08,61.299999,61.580002,60.840000,61.009998,21220800,1.549141,UP,1
2016-12-09,61.180000,61.990002,61.130001,61.970001,27349400,0.321694,UP,1
2016-12-12,61.820000,62.299999,61.720001,62.169998,20198100,1.286125,UP,1
2016-12-13,62.500000,63.419998,62.240002,62.980000,35718900,-0.478620,DOWN,-1
2016-12-14,63.000000,63.450001,62.529999,62.680000,30352700,-0.159793,DOWN,-1
...,...,...,...,...,...,...,...,...
2024-07-23,443.899994,448.390015,443.100006,444.850006,13107100,-3.718818,DOWN,-1
2024-07-24,440.450012,441.480011,427.589996,428.899994,26805800,-2.509560,DOWN,-1
2024-07-25,428.799988,429.799988,417.510010,418.399994,29943800,1.615443,UP,1


In [12]:
df[['change_tommorow_direction']]

,change_tommorow_direction
Date,
2016-12-08,UP
2016-12-09,UP
2016-12-12,UP
2016-12-13,DOWN
2016-12-14,DOWN
...,...
2024-07-23,DOWN
2024-07-24,DOWN
2024-07-25,UP


##### Buy if it goes and sell if down

> You can only sell if you have already bought

In [23]:
list_buy_sell = []
already_bought = False

In [24]:
for tommorow_direction in df.change_tommorow_direction:
  if tommorow_direction == 'UP' and already_bought == False:
    list_buy_sell.append(1)
    already_bought = True
  elif tommorow_direction == 'DOWN' and already_bought == True:
    list_buy_sell.append(-1)
    already_bought = False
  else:
    list_buy_sell.append(0)

In [25]:
df['buy_sell_track'] = list_buy_sell
df

,Open,High,Low,Close,Volume,change_tommorow,change_tommorow_direction,buy_sell,buy_sell_track
Date,,,,,,,,,
2016-12-08,61.299999,61.580002,60.840000,61.009998,21220800,1.549141,UP,1,1
2016-12-09,61.180000,61.990002,61.130001,61.970001,27349400,0.321694,UP,1,0
2016-12-12,61.820000,62.299999,61.720001,62.169998,20198100,1.286125,UP,1,0
2016-12-13,62.500000,63.419998,62.240002,62.980000,35718900,-0.478620,DOWN,-1,-1
2016-12-14,63.000000,63.450001,62.529999,62.680000,30352700,-0.159793,DOWN,-1,0
...,...,...,...,...,...,...,...,...,...
2024-07-23,443.899994,448.390015,443.100006,444.850006,13107100,-3.718818,DOWN,-1,-1
2024-07-24,440.450012,441.480011,427.589996,428.899994,26805800,-2.509560,DOWN,-1,0
2024-07-25,428.799988,429.799988,417.510010,418.399994,29943800,1.615443,UP,1,1


#### Add conditions to the strategy

In [27]:
class ClassificationUP(Strategy):
    def init(self):
        self.model = model_dt
        self.already_bought = False

    def next(self):
        explanatory_today = self.data.df.iloc[[-1],:]
        forecast_tomorrow = model_dt.predict(explanatory_today)[0]
        
        # conditions to sell or buy
        if forecast_tomorrow == 'UP' and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow == 'DOWN' and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

### Define initial conditions

In [28]:
bt = Backtest(data=df_explanatory, strategy=ClassificationUP,
              cash=10000, commission=.002, exclusive_orders=True)

### Run backtesting

In [30]:
results = bt.run()

### Interpret backtesting results

In [31]:
results.to_frame(name='Values').loc[:'Return [%]']

,Values
Start,2016-12-08 00:00:00
End,2024-07-29 00:00:00
Duration,2790 days 00:00:00
Exposure Time [%],99.895833
Equity Final [$],587781680.321286
Equity Peak [$],587781680.321286
Return [%],5877716.803213


## Practice to master the knowledge

Work on the challenge with another dataset:

1. Learn the <a>mental models</a> to solve the challenge faster.
2. Complete the <a href="03C_Backtesting ML Classification-Based.ipynb">notebook</a>.